# Run Agent in Environment with different Algorithms

In [ ]:
# Basics
import os
import warnings

# Data 
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import numpy as np

# Logging
import logging
import wandb
from wandb.integration.sb3 import WandbCallback

# Algorithms 
from stable_baselines3 import A2C
from stable_baselines3 import SAC
from stable_baselines3 import DDPG
from stable_baselines3 import PPO
from stable_baselines3 import TD3
from sb3_contrib import RecurrentPPO
from sb3_contrib import TQC
from sb3_contrib import TRPO
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env

import torch
import torch.nn as nn
from rl_zoo3 import linear_schedule

from gym import make
from gym.wrappers import RecordEpisodeStatistics

import subprocess
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from vpp_gym.vpp_gym.utils.register_env import register_env

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env


## Seed Env

In [ ]:
SEED = 44

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Register the Environment 

In [ ]:
CONFIG_NAME = "vpp_config_1_training.json"

register_env(config=CONFIG_NAME, seed=SEED)

## Test the environment

In [ ]:
# It will check your custom environment and output additional warnings if needed
env_to_check = make('VPPBiddingEnv-TEST-v1', render_mode=None)
check_env(env_to_check)
env_to_check.close()

# Stable Baselines

### Offline Training and later sync logs

In [ ]:
os.environ["WANDB_API_KEY"] = "0cea1eee5f42654eca0de365f0acca116367c9b4"
os.environ["WANDB_MODE"] = "offline"

## Training Function

In [ ]:
def train_algorithm(algorithm): 
    env = make('VPPBiddingEnv-TRAIN-v1', render_mode="human")
    #env = make('VPPBiddingEnv-TUNING-logs-v1')
    env = Monitor(env) 
    env = RecordEpisodeStatistics(env)
    
    if algorithm == "R_PPO": 
        policy = 'MultiInputLstmPolicy'
    else: 
        policy = 'MultiInputPolicy'

    wandb.init(
        sync_tensorboard=True, 
        project="RL-VPP-Training",
        save_code=True,
        entity="jlu237", 
        tags=[algorithm] + EXPERIMENT_TAGS, 
        job_type="training"
    )
    
    model_params = HYPERPARAMS[algorithm]
    model = ALGORITHMS[algorithm](policy, env, verbose=0,  seed = SEED , **model_params)
    model.learn(total_timesteps=EXPERIMENT_TIMESTEPS,
                log_interval=1,
                progress_bar = True,
                callback=WandbCallback(
                    gradient_save_freq=1,
                    verbose=0))
    wandb.finish()
    return_code = subprocess.run("wandb sync wandb/latest-run", shell=True)
    
    return return_code, model 


## Evaluate Function

In [ ]:
def evaluate_algorithm(algorithm, model):
    eval_env = make('VPPBiddingEnv-EVAL-v1', render_mode="human")
    eval_env = Monitor(eval_env) 
    eval_env = RecordEpisodeStatistics(eval_env) 

    wandb.init(
        sync_tensorboard=True, 
        project="RL-VPP-Evaluation",
        save_code=True,
        entity="jlu237", 
        tags=[algorithm] + EXPERIMENT_TAGS, 
        job_type="eval",
    )

    episodes = 70
    for i_episode in range(episodes):
        observation = eval_env.reset()
        if algorithm == "R_PPO":
            lstm_states = None
            num_envs = 1
            # Episode start signals are used to reset the lstm states
            episode_starts = np.ones((num_envs,), dtype=bool)
            for t in range(1):
                action, lstm_states = model.predict(observation, state=lstm_states, episode_start=episode_starts, deterministic=True)
                observation, reward, dones, info = eval_env.step(action)
                episode_starts = dones
        else: 
            for t in range(1):
                action, _states = model.predict(observation, deterministic = True)
                observation, reward, done, info = eval_env.step(action)

    eval_env.close()
    wandb.finish()
    return_code = subprocess.run("wandb sync wandb/latest-run", shell=True)
    return return_code


In [ ]:
# activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU, "elu": nn.ELU, "leaky_relu": nn.LeakyReLU}[activation_fn]
#  NormalActionNoise(mean=[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.],
#                                              sigma=[0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106,0.15087106]
#                                             )

HYPERPARAMS = {
    
    "A2C": {},
    
    "DDPG": {},
    
    "SAC": {
            'gradient_steps': -1,
           },
    
    "PPO": {},
    
    "TD3": {},
    
    #"DQN": {},
    
    # SB3 Contrib,
    "TQC": {},
    
    "TRPO": {'learning_rate': 0.00006428106794995492,
             'n_steps': 8,
             'batch_size': 8,
             'gamma': 0.95,
             'cg_max_steps': 10,
             'cg_damping': 0.01,
             'line_search_shrinking_factor': 0.8,
             'line_search_max_iter': 5,
             'n_critic_updates': 25,
             'gae_lambda': 0.98,
             'normalize_advantage': False,
             'use_sde': True,
             'target_kl': 0.001,
             'policy_kwargs': {'net_arch': [{'pi': [64, 64], 'vf': [64, 64]}],
                               'ortho_init': True,
                               'activation_fn': nn.Tanh,
                               'log_std_init': -1.882337250753788,
                               'full_std': False,
                               'sde_net_arch': [64, 64]},
             'sde_sample_freq': 0},
    
    "R_PPO": {'learning_rate': 0.0018671792141252545,
              'n_steps': 10,
              'batch_size': 10,
              'n_epochs': 5,
              'gamma': 0.9999,
              'gae_lambda': 0.9,
              'clip_range': 0.2,
              'normalize_advantage': True,
              'ent_coef': 3.180891783226621e-06,
              'vf_coef': 0.8521929103610445,
              'max_grad_norm': 0.9,
              'target_kl': 0.001,
              'policy_kwargs': {'net_arch': [{'pi': [64, 64], 'vf': [64, 64]}],
                                'full_std': False,
                                'activation_fn': nn.ELU,
                                'ortho_init': False}}
            }


In [ ]:
ALGORITHMS = {
    "A2C": A2C,
    "DDPG": DDPG,
    "PPO": PPO,
    "SAC": SAC,
    "TD3": TD3,
    #"DQN": DQN,
    # SB3 Contrib,
    "TQC": TQC,
    "TRPO": TRPO,
    "R_PPO": RecurrentPPO,
}

In [ ]:
EXPERIMENT_TAGS = ["config1_tuning", 
                   "AS: Box(12,)", 
                   "2785",
                   "S44",
                   "OS: Dict(7 features)"
                  ]

EXPERIMENT_TIMESTEPS = 2785  #2785 #5570  #11140


## Train all Algorithms

## Single Algorithm Training

In [ ]:
ALGORITHM = "PPO"

# MultiDiscrete 2: A2C, PPO, R_PPO, TRPO, M_PPO
# MultiDiscrete 1: funktionierte nicht mit A2C und R_PPO, aber evtl mit PPO, TRPO ? 

print("now training " + ALGORITHM)
return_code, model = train_algorithm(ALGORITHM) 
print("training finished with : " + str(return_code))

## Single Algorithm Evaluation

In [ ]:
return_code = evaluate_algorithm(ALGORITHM, model)
print("evaluation finished with : " + str(return_code))